# Terms creation in 2P-Kt

## Goals

- Learn how to create/inspect terms in `2p-kt`
- Understand the representation of lists in logic

## Terms hierarchy in `2p-kt`

Simplified view of the type hierarchy of `Term` in `2p-kt`:
![Type hierarchy of `Term`](http://www.plantuml.com/plantuml/svg/PP11IiOm48NtSufSuB-1Yugu4CI5YlkXdRR1P5ec4qJmy1fYoBHTlk-HyBuPag9eZW7If-ST1QDBAqxRb-V5pHWI0NXMaXb7SkyaqSI7ZPCYgq4VA2QzgUJHrYyvalZHa2TMMiW-UYKC9jxh5kq4oRgQ_1Xt_NxsIGqpiSq4ADlTtS_jpurlRTa1DfypBaxGkewRowMfa-6_xxY8NMax0DH9frzEiAf1gJgh9u5ITTZlSkJr60Fnb6Fu1G00)


## Preliminaries

Let's first import `2p-kt` (`%use` directive only work in Jupyter):

In [ ]:
%use 2p-kt@/ktlibs
%use kt-math@/ktlibs
import it.unibo.tuprolog.core.List as LogicList

Let's create a parser for converting strings into terms:

In [2]:
val parser = TermParser.withNoOperator()

Let's create a formatter for converting terms into human-readable strings:

In [3]:
val formatter = TermFormatter.prettyVariables()

## Atoms

Atoms are alphanumerics constants.
When
- containing spaces, 
- or beginning with uppercase letters or undescores

atom are represented as wrapped within single/double apices.

In [4]:
val representations = listOf(
    "anAtom", 
    "'an atom with spaces'",
    "\"ACapitalAtom\""
)
representations

[anAtom, 'an atom with spaces', "ACapitalAtom"]

Atoms can be programmatically constructed by means of the `Atom.of` factory method, accepting a `String` *without apices* as input.

Otherwise, atoms can be parsed from a `String` by means of a `TermParser`.

Below, try to construct the atom **programmatically**, out of a representation, in order for it to match the atom parsed from the same representation.

In [5]:
for (repr in representations) {
//     val term: Term = TODO("create an ${Atom::class.simpleName} term whose representation is equal to $repr")
    val term: Term = Atom.of(repr.replace("'", "").replace("\"", ""))
    val expected = parser.parseTerm(repr)

    assertEquals(expected, term)
    assertTrue(term is Struct)
    assertTrue(term is Constant)
    assertTrue(term is Atom)
    assertTrue(term.isGround)
    assertEquals(repr.replace("'", "").replace("\"", ""), term.value)
    assertEquals(repr.replace("\"", "'"), formatter.format(term))
}

## Numbers

Numbers are numeric constants.
They can either be integer or real.

### Integers

Integers are numbers with sign, with no decimal part, and with no limit in representation size.

In [6]:
val representations = listOf("1", "2", "0", "-1", "10000000000000000000")
representations

[1, 2, 0, -1, 10000000000000000000]

Integers can be programmatically constructed by means of the `Integer.of` factory method, accepting a `String`, `Int`, `Long`, or `BigInteger`.

Otherwise, integers can be parsed from a `String` by means of a `TermParser`.

Below, try to construct the integer **programmatically**, out of a representation, in order for it to match the integer parsed from the same representation.

In [7]:
for (repr in representations) {
//     val term: Term = TODO("create an ${Integer::class.simpleName} term whose value is equal to $repr")
    val term = Integer.of(repr)
    val expected = parser.parseTerm(repr)

    assertEquals(expected, term)
    assertTrue(term is Numeric)
    assertTrue(term is Constant)
    assertTrue(term is Integer)
    assertTrue(term.isGround)
    assertEquals(BigInteger.of(repr), term.value)
    assertEquals(repr, formatter.format(term))
}

### Reals

Reals are numbers with sign, with an unlimited-precision decimal part, and with no limit in representation size.


In [8]:
val representations = listOf("1.2", "-3.4", "0.0", BigDecimal.PI.toString())
println(representations)

[1.2, -3.4, 0.0, 3.141592653589793238462643383279502884197169399375105820974944592307816406286208998628034825342117067982148086513282306647093844609550582231725359408128481117450284102701938521105559644622948954930381964428810975665933446128475648233786783165271201909145648]


Reals can be programmatically constructed by means of the `Real.of` factory method, accepting a `String`, `Float`, `Double`, or `BigDecimal`.

Otherwise, reals can be parsed from a `String` by means of a `TermParser`.

Below, try to construct the real **programmatically**, out of a representation, in order for it to match the real parsed from the same representation.

In [9]:
for (repr in representations) {
//     val term: Term = TODO("Create a ${Real::class.simpleName} term whose representation is equal to $repr")
    val term = Real.of(repr)
    val expected = parser.parseTerm(repr)

    assertEquals(expected, term)
    assertTrue(term is Numeric)
    assertTrue(term is Constant)
    assertTrue(term is Real)
    assertTrue(term.isGround)
    assertEquals(BigDecimal.of(repr), term.value)
    assertEquals(repr, formatter.format(term))
}
    

## Variables

Variables are (possibly, *named*) placeholders for unknown terms.
Variables whose name *begins with* an underscore are considered __anonymous__.
Names are commonly __uppercase__.

In [10]:
val names = listOf("A", "B", "_", "_A", "_B", "SomeVariable")
names

[A, B, _, _A, _B, SomeVariable]

Variables can be programmatically constructed by means of the `Var.of` factory method, accepting a `String`.

Otherwise, variables can be parsed from a `String` by means of a `TermParser`.

> Each new variable is always created different from any other variable created before it, **even if the name is the same**.

Below, try to construct the variable **programmatically**, out of a representation, in order for it to match the variable parsed from the same representation.

In [11]:
for (name in names) {
//     val term: Term = TODO("Create a ${Var::class.simpleName}iable term whose name is equal to $name")
    val term = Var.of(name)
    val expected = parser.parseTerm(name)

    assertNotEquals(expected, term) // notice this!
    assertTrue(expected.equals(term, useVarCompleteName = false))
    assertTrue(term is Var)
    assertFalse(term.isGround)
    assertEquals(name, term.name)
    assertEquals(name, formatter.format(term))
    assertTrue(term.completeName.startsWith(name + "_"))
    assertEquals(name == "_", term.isAnonymous)
}

## Structures

Structures are named records with zero, one, or more terms as **arguments**.
The name is called **functor**. 
The amount of arguments is called **arity**.
Atoms are essentially structures with 0 arguments.

In [12]:
val representation = "person(giovanni, ciatto, 31)"

Structures can be programmatically constructed by means of the `Struct.of` factory method, accepting a `String` and an arbitrary amount of `Term`s.

Otherwise, structures can be parsed from a `String` by means of a `TermParser`.

Below, try to construct the structure **programmatically**, out of a representation, in order for it to match the structure parsed from the same representation.

In [13]:
// val term: Term = TODO("Create a ${Struct::class.simpleName}ured term whose representation is equal to $representation")
val term = Struct.of("person", Atom.of("giovanni"), Atom.of("ciatto"), Integer.of(31))
val expected = parser.parseTerm(representation)

assertEquals(expected, term)
assertTrue(term is Struct)
assertEquals("person", term.functor)
assertEquals(3, term.arity)
assertEquals(
    listOf(Atom.of("giovanni"), Atom.of("ciatto"), Integer.of(31)),
    term.args
)
assertEquals(term[0], Atom.of("giovanni"))
assertEquals(term[1], Atom.of("ciatto"))
assertEquals(term[2], Integer.of(31))
assertTrue(term.isGround)
assertEquals(representation, formatter.format(term))

Further arguments may be added to a structure.
In that case **a new structure is created** and the original one is unaffected.

In [14]:
val newTerm = term.addLast(Var.of("DateOfBirth"))
val newRepresentation = "person(giovanni, ciatto, 31, DateOfBirth)"
val newExpected = parser.parseTerm(newRepresentation)

assertNotSame(term, newTerm)
assertTrue(newExpected.equals(newTerm, useVarCompleteName = false))
assertFalse(newTerm.isGround)

> All sub-types of `Term` are **immutable** in `2p-kt`.
> This means that all editing functionalities work by creating copies of the input term under the hood.

## Lists

Logic lists are essentially immatable single-linked lists.
They are implemented as structures of two types:
- the couple `.(Head, Tail)`, most often denoted as `[Head | Tail]`
- the empty list `[]` (which is technically an atom)

Longer lists can be achieved by means of recursion.

When the innermost, righmost `Tail` is an empty list, then it is omitted from the notation:
```prolog
[1, 2, 3] = [1, 2, 3 | []] = .(1, .(2, .(3, [])))
```

In [15]:
val representation = "[1, a, f(x)]"

Lists can be programmatically constructed in several ways:
- `LogicList.of(...)` which accepts the items of the list, and assumes the tail to be the empty list
- `LogicList.from(..., last=...)` which accepts the items of the list, and lets the callee specify the tail
- `Cons.of(head, tail)` which creates a couple
- `EmptyList()` which creates the empty list

Otherwise, lists can be parsed from a `String` by means of a `TermParser`.

Below, try to construct the list **programmatically**, out of a representation, in order for it to match the list parsed from the same representation.

In [ ]:
val term = LogicList.of(Integer.of(1), Atom.of("a"), Struct.of("f", Atom.of("c")))
val expected = parser.parseTerm(representation)

assertEquals(expected, term)
assertTrue(term is Struct)
assertTrue(term is List)
assertTrue(term is Cons)
assertEquals(".", term.functor)
assertEquals(2, term.arity)
assertEquals(term[0], Integer.of(1))
assertEquals(term[1], LogicList.of(Atom.of("a"), Struct.of("f", Atom.of("x"))))
assertEquals(representation, formatter.format(term))

assertEquals(
    actual = term,
    expected = Cons.of(
        head = Integer.of(1),
        tail = Cons.of(
            head = Atom.of("a"),
            tail = Cons.of(
                head = Struct.of("f", Atom.of("x")),
                tail = EmptyList.instance
            )
        )
    )
)